# Partycjonowanie i glooming

* Kiedy RDD jest tworzone, możesz określić liczbę partycji
* Defaultowo jest liczbą executorów definiowaną podczas ustawienia SparkContext

In [3]:
from pyspark import SparkContext
sc = SparkContext()

In [4]:
A=sc.parallelize(range(1000000))
print(A.getNumPartitions())

4


## Repartycjonowanie 

In [5]:
D = A.repartition(10)
print(D.getNumPartitions())

10


Można również zdefiniować liczbę partycji podczas tworzenia RDD

In [8]:
A = sc.parallelize(range(1000000),numSlices=10)
print(A.getNumPartitions())

10


## Partycjonowanie w celu równoważenia obciążenia

Załóżmy, że zaczynamy od 10 partycji, wszystkie z dokładnie taką samą liczbą elementów

In [11]:
A=sc.parallelize(range(1000000))\
    .map(lambda x:(x,x)).partitionBy(10)
print(A.glom().map(len).collect())

[100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000, 100000]


* Załóżmy że używamy funkcji filter w celu selekcji pewnych elementów w zbiorze A
* Pewnie partycje mogą mieć więcej elementów niż inne 

In [12]:
#select 10% of the entries
B=A.filter(lambda pair: pair[0]%5==0)
# get no. of partitions
print(B.glom().map(len).collect())

[100000, 0, 0, 0, 0, 100000, 0, 0, 0, 0]


* Przyszłe operacje na zbiorze B będą używane tylko na dwóch workerach
* **Inne workery nie będą robiły nic** ze względu na to że ich partycje będą puste

### Jak można temu zapobiec?

* W celu efektywnego przetwarzania należy zrepartycjonować RDD 
* Jednym ze sposobów jest repartycjonowanie z wykorzystaniem klucza 

* Metoda .partitionBy(k) przyjmuje (klucz,wartość) RDD gdzie klucze są integer.
* Partycjonowanie RDD na k partycji
* Element (key,value) jest umieszczany w key % k

In [14]:
C=B.map(lambda pair:(pair[1]/10,pair[1])).partitionBy(10) 
print(C.glom().map(len).collect())

[20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000]


### Inny sposób  

Innym sposobem jest użycie automatycznego partycjonowania wykorzystując repartition(k)\

* Zaletą rozwiązania jest brak konieczności definiowania klucza
* Wadą jest brak kontroli na partycjonowaniem

In [15]:
C=B.repartition(10)
print(C.glom().map(len).collect())

[20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000]


## Glom()

* **glom()** transformuje każdą partycję na tuple(immutable list) 
* Tworzy RDD zawierające tuple
* workery mogą odnosić się do elementów po indeksie partycji
* dalej nie możesz assignować wartości do poszczególnych elementów, RDD dalej jest immutable

In [17]:
print(C.glom().map(len).collect())

[20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000, 20000]
